<a href="https://colab.research.google.com/github/francescoperagine/EyeInTheSkyNotebooks/blob/main/EyeInTheSky_VisDrone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

# Now install your packages
!pip install loguru==0.7.3 \
             roboflow==1.1.53 \
             tqdm==4.67.1 \
             typer==0.15.1 \
             ultralytics==8.3.68 \
             wandb==0.19.6

In [ ]:
from roboflow import Roboflow
import torch
from loguru import logger
import wandb
from google.colab import userdata
from ultralytics import YOLO, checks, settings
import time
from google.colab import files

In [ ]:
model_name = userdata.get("MODEL_NAME")
api_key = userdata.get("WANDB_APY_KEY")

project_name = "EyeInTheSky"
dataset = "VisDrone"

In [ ]:
settings.update({"wandb": True})

In [ ]:
wandb.login(key=api_key)

In [ ]:
try:
    device =  0 if torch.cuda.is_available() else "cpu"
except Exception as e:
    print(f"Error setting device: {e}")
print(f"Device: {device}")

In [ ]:
model = YOLO(f"{model_name}.pt")
results = model.train(data=f"{dataset}.yaml",
    freeze=21,
    amp=True,
    epochs=100,
    imgsz=640,
    patience=5,
    project=project_name,
    name=dataset
  )

logger.info("Performing training for model...")
logger.info(checks())

In [ ]:
print(results)

In [ ]:
# model = YOLO(f"{model_name}.pt")
# results = model.train(data=f"{dataset}.yaml",
#     freeze=22,
#     amp=True,
#     epochs=100,
#     imgsz=640,
#     patience=5,
#     project=project_name,
#     name=dataset
#   )

In [ ]:
metrics = model.val()

In [ ]:
model_path = model.export()
wandb.save(model_path)

In [ ]:
timestamp = time.strftime("%Y%m%d_%H%M%S")  # Get current time in YYYYMMDD_HHMMSS format
zip_filename = f"/content/{project_name}_runs_{timestamp}.zip"

!zip -r "$zip_filename" /content/"$project_name"

In [ ]:
files.download(zip_filename)